In [ ]:
%load_ext autoreload
%aimport rasterio_helpers
%aimport proj_scope_vars
%autoreload 1

import importlib 

import pdal
import glob
import json
import os
from rasterio_helpers import *
from proj_scope_vars import *
import rasterio
from rasterio.warp import reproject
import numpy as np


In [ ]:
#This creates a TIF of the height above ground using the delaunay cal.
def laz_to_height(lazfile_in, tiffile_out):
    pdal_json = {
         "pipeline": [
             lazfile_in,
             {
                 # Delaunay causes silent failures, at least on the machine I'm running
                 "type":"filters.hag_delaunay"
                 #"type":"filters.hag_nn"
             },
             {

                 "filename": tiffile_out,
                 "gdaldriver": "GTiff",
                 "output_type": "mean",
                 "dimension" : "HeightAboveGround",
                 "window_size" : 5,
                 "resolution": "1",
                 "type": "writers.gdal"
             }
         ]
     }
    pdal_json_str = json.dumps(pdal_json)
    pipeline = pdal.Pipeline(pdal_json_str)
    count = pipeline.execute()


In [ ]:
#This creates a TIF of the lidar data
def laz_to_tif(lazfile_in, tiffile_out, out_type="all"):
    pdal_json = {
         "pipeline": [
             lazfile_in,
             {

                 "filename": tiffile_out,
                 "gdaldriver": "GTiff",
                 "output_type": out_type,
                 "window_size" : 5,
                 "resolution": "1",
                 "type": "writers.gdal"
             }
         ]
     }
    pdal_json_str = json.dumps(pdal_json)
    pipeline = pdal.Pipeline(pdal_json_str)
    count = pipeline.execute()

In [ ]:
#Iterates through all the TIFFs, calls laz_to_height, and merges the results
# into one large TIFF with a name defined by lidar_height_merged_tif
lazfiles=glob.glob(lidar_dl_dir + '/*laz')
if not os.path.exists(lidar_dl_tif_dir):
    os.makedirs(lidar_dl_tif_dir)



for lazfile in lazfiles:
    filename = os.path.basename(lazfile)
    print(filename)
    outfile = lidar_dl_tif_dir + '/' + os.path.splitext(filename)[0] + '_min_max.tif'
    laz_to_tif(lazfile, outfile, 'min, max')
    print(outfile)




In [ ]:
#This merges the dataset into a single TIFF File
lidar_datasets = []

tiffiles=glob.glob(lidar_dl_tif_dir + '/*_min_max.tif')
for tiffile in tiffiles:
    ds = rasterio.open(tiffile)
    lidar_datasets.append(ds)
    
lidar_merged_ds = merge_ds(lidar_datasets)

print(lidar_merged_ds.profile)
show(lidar_merged_ds)


ld_rtns=['min', 'max']    
ld_nps = {}

lidar_profile = lidar_merged_ds.profile
lidar_nodata  = lidar_merged_ds.nodata
band = 1
for ld_rtn in ld_rtns:
    #Convert to dtm_crs: epsg(32610)
    lidar_np,lidar_trans = reproject(rasterio.band(lidar_merged_ds, band), 
                                        dst_nodata=lidar_nodata, 
                                        dst_crs=rasterio.CRS.from_epsg(epsg), 
                                        dst_resolution=(1,1))
    
    ld_nps[ld_rtn] = lidar_np
       
    band += 1


lidar_profile.update( {"driver": "GTiff",
                     "nodata" : lidar_nodata,
                     "crs" : rasterio.CRS.from_epsg(epsg),
                     "height": lidar_np.shape[1],
                     "width": lidar_np.shape[2],
                     "count": 1,
                     "transform": lidar_trans})

print(lidar_profile)


for ld_rtn in ld_rtns:
    ld_rtn_ds = get_ds(ld_nps[ld_rtn], lidar_profile)
    
    rp_tif = lidar_dl_tif_dir + '/lidar_sf_merged_epsg32610_' + ld_rtn + '.tif'
    print("Repojected return:" + ld_rtn)
    print(ld_rtn_ds.profile)
    show(ld_rtn_ds)
    
    save_ds(ld_rtn_ds, rp_tif)
    

In [ ]:
#This computes the height.
height_np = np.where(np.logical_or((ld_nps['max']==lidar_nodata), (ld_nps['max']==lidar_nodata)),
                     lidar_nodata, 
                     ld_nps['max']-ld_nps['min'])

    
height_ds = get_ds(height_np, lidar_profile)

In [ ]:
height_masked_ds = mask_ds(height_ds, studyarea, studyarealayer)

save_ds(height_masked_ds, height_map_tif)

In [ ]:
show(height_masked_ds)

In [ ]:
print(height_map_tif)

In [ ]:
#Creates 

lidar_height_merged_ds = rasterio.open(lidar_height_merged_tif)

height_profile = lidar_height_merged_ds.profile
height_nodata  = lidar_height_merged_ds.nodata


#Convert to dtm_crs: epsg(32610)
height_np,heigth_trans = reproject(rasterio.band(lidar_height_merged_ds, 1), 
                                    dst_nodata=height_nodata, 
                                    dst_crs=rasterio.CRS.from_epsg(epsg), 
                                    dst_resolution=(1,1))

height_profile.update({"driver": "GTiff",
                 "nodata" : height_nodata,
                 "crs" : rasterio.CRS.from_epsg(epsg),
                 "height": height_np.shape[1],
                 "width": height_np.shape[2],
                 "transform": heigth_trans})


show(heigth_ds)

save_ds(heigth_ds, height_map_tif)

In [ ]:
#Iterates through all the TIFFs, calls laz_to_height, and merges the results
# into one large TIFF with a name defined by lidar_height_merged_tif
lazfiles=glob.glob(lidar_dl_dir + '/*laz')
if not os.path.exists(lidar_dl_tif_dir):
    os.makedirs(lidar_dl_tif_dir)



for lazfile in lazfiles:
    filename = os.path.basename(lazfile)
    print(filename)
    outfile = lidar_dl_tif_dir + '/' + os.path.splitext(filename)[0] + '_hag_delaunay.tif'
    laz_to_height(lazfile, outfile)
    print(outfile)




In [ ]:
lidar_datasets = []

tiffiles=glob.glob(lidar_dl_tif_dir + '/*_hag_delaunay.tif')
for tiffile in tiffiles:
    ds = rasterio.open(tiffile)
    lidar_datasets.append(ds)
    
lidar_merged_ds = merge_ds(lidar_datasets)

lidar_profile = lidar_merged_ds.profile
lidar_nodata  = lidar_merged_ds.nodata

lidar_np,lidar_trans = reproject(rasterio.band(lidar_merged_ds, 1), 
                                    dst_nodata=lidar_nodata, 
                                    dst_crs=rasterio.CRS.from_epsg(epsg), 
                                    dst_resolution=(1,1))

lidar_profile.update( {"driver": "GTiff",
                     "nodata" : lidar_nodata,
                     "crs" : rasterio.CRS.from_epsg(epsg),
                     "height": lidar_np.shape[1],
                     "width": lidar_np.shape[2],
                     "transform": lidar_trans})

height_ds = get_ds(lidar_np, lidar_profile)
hd_height_map_tif = os.path.splitext(height_map_tif)[0] + '_hag_delaunay.tif'


In [ ]:
height_masked_ds = mask_ds(height_ds, studyarea, studyarealayer)
save_ds(height_masked_ds, hd_height_map_tif)
show(height_masked_ds)
print(lidar_profile)

In [ ]:
delaunay_height_map_tif = os.path.splitext(height_map_tif)[0] + '_hag_delaunay.tif'
first_last_height_map_tif = height_map_tif
print(height_map_tif)
max_height_map_tif = os.path.splitext(height_map_tif)[0] + '_max_height.tif'
print(delaunay_height_map_tif)


firts_last_ds = rasterio.open(first_last_height_map_tif)
first_last_np = firts_last_ds.read()
delaunay_ds = rasterio.open(delaunay_height_map_tif)
delaunay_np = delaunay_ds.read()


max_height_np = np.maximum(delaunay_np, first_last_np)
max_height_profile = delaunay_ds.profile
print(max_height_profile)
print(max_height_np.shape)

max_height_ds = get_ds(max_height_np, max_height_profile)
save_ds(max_height_ds, max_height_map_tif)

In [ ]:
import fiona
with fiona.open(studyarea, layer=studyarealayer) as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]  
print(studyarea)
from shapely.geometry import shape
multi = shape(shapes[0])
print(multi)
print(dir(multi))
print(multi.convex_hull)

In [ ]:
import pprint

with fiona.open(studyarea, layer=studyarealayer) as src:
    pprint.pprint(src[1].schema)


# with fiona.open(, layer=studyarealayer)  as layer:
#     for feature in layer:
#         print(feature['geometry'])

In [ ]:
pprint.pprint(shapefile)